In [1]:
import torch
from transformers import BertModel, BertConfig, BertTokenizer

In [4]:
model = BertModel.from_pretrained('bert-base-uncased')  # Используйте нужную вам версию BERT

# Загрузка весов модели из вашего файла .pth
model_path = 'bert_classifier.pth'
state_dict = torch.load(model_path)

# Замена ключей в state_dict
new_state_dict = {}
for key in state_dict.keys():
    new_key = key.replace("bert.", "")  # убираем префикс "bert."
    new_state_dict[new_key] = state_dict[key]

new_state_dict = {k: v for k, v in new_state_dict.items() if "fc" not in k}

# Загрузка весов в модель
model.load_state_dict(new_state_dict)

<All keys matched successfully>

In [5]:
def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
    
    return "positive" if preds.item() == 1 else "negative"

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [8]:
model.to(device)
test_text = "Worst movie of the year."
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print("Worst movie of the year.")
print(f"Predicted sentiment: {sentiment}")

TypeError: max() received an invalid combination of arguments - got (BaseModelOutputWithPoolingAndCrossAttentions, dim=int), but expected one of:
 * (Tensor input, *, Tensor out)
 * (Tensor input, Tensor other, *, Tensor out)
 * (Tensor input, int dim, bool keepdim, *, tuple of Tensors out)
 * (Tensor input, name dim, bool keepdim, *, tuple of Tensors out)
